Importing necessary libraries

In [7]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from Object_Tracking import *
import time
import os
import subprocess
import IPython.display as ipd

Loading the custom trained model

In [8]:
# best.pt is copied from "speed_source_code/model_training/runs/detect/train11/weights/best.pt"
model = YOLO('best.pt')

Help function to get coordinates of a particular point on the screen


In [9]:
def Vehicle_Speed_Detection(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)

Using one test video(MVI_40981) to demonstrate speed and object detection


In [10]:
image_folder = 'MVI_40981'
image_files = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith(('.jpg'))]
image_files.sort()

Intializing variables

In [11]:
count = 0
fps = 30

tracker = Tracker()

cy1=206
cy2=409
offset=6

# Frame dimensions
width = 1400
height = 850

vh_id_time = {}
counter = []
a_speed_kh=0

Speed Detection, Video Compilation and generating speed_result.txt

In [12]:
cv2.namedWindow('Vehicle_Speed_Detection')        
cv2.setMouseCallback('Vehicle_Speed_Detection', Vehicle_Speed_Detection)

output_file = open("speed_result.txt", "w")# Creating a txt file

# Initializing variables to make video from the frames
output_video_path = 'test_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Interating over each frame in the folder
for image_file in image_files:
    
    frame = cv2.imread(image_file)
    count += 1
    frame = cv2.resize(frame, (width, height))

    # Object detection
    results = model.predict(frame)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")

    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        # Adding bounding boxes to detected objects
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2)
        cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)

        if cy2 < (cy + offset) and cy2 > (cy - offset):
            vh_id_time[id] = time.time()
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

        # Calculating speed of the vehicle between line1 and line2 
        if id in vh_id_time:
            if cy1 < (cy + offset) and cy1 > (cy - offset):
                elapsed_time = time.time() - vh_id_time[id]
                if counter.count(id) == 0:
                    counter.append(id)
                    distance = 10  #meters
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6
                    cv2.putText(frame, str(int(a_speed_kh)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
            cv2.putText(frame,('Vehicle ID : ')+ str(id),(27,765),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,223,255),2) 
    speed_result = f"{'MVI_40981'} {count} {id} {x3} {y3} {x4} {y4} {int(a_speed_kh)} {results[0].boxes.conf} "
    output_file.write(speed_result + "\n")

    # Defining Region of intrest (line1 and line2)
    cv2.line(frame,(239,cy1),(1231,cy1),(255,255,255),1)
    cv2.putText(frame,('LINE1'),(237,198),cv2.FONT_HERSHEY_COMPLEX,0.8,(225,225,255),2)

    cv2.line(frame,(0,cy2),(1355,cy2),(255,255,255),1)
    cv2.putText(frame,('LINE2'),(42,405),cv2.FONT_HERSHEY_COMPLEX,0.8,(225,225,255),2)

    cv2.putText(frame, ('Speed of the Vehicle : ') + str(int(a_speed_kh)) + 'Km/h', (36,821), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 223, 255), 2)
    cv2.imshow("Vehicle_Speed_Detection", frame)

    # Compilation of all the frames into video
    video_writer.write(frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

video_writer.release()
output_file.close()
cv2.destroyAllWindows()


0: 160x224 2 cars, 37.3ms
Speed: 4.3ms preprocess, 37.3ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 2 cars, 22.9ms
Speed: 1.1ms preprocess, 22.9ms inference, 0.8ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 2 cars, 27.3ms
Speed: 2.4ms preprocess, 27.3ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 3 cars, 28.1ms
Speed: 0.8ms preprocess, 28.1ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 3 cars, 24.1ms
Speed: 2.2ms preprocess, 24.1ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 3 cars, 24.6ms
Speed: 0.5ms preprocess, 24.6ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 3 cars, 26.7ms
Speed: 0.7ms preprocess, 26.7ms inference, 1.3ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 3 cars, 28.5ms
Speed: 0.7ms preprocess, 28.5ms inference, 1.5ms postprocess per image at shape (1, 3, 160, 224)

0: 160x

[82, 24]
[87, 15]
[89, 9]



0: 160x224 4 cars, 29.7ms
Speed: 0.7ms preprocess, 29.7ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 4 cars, 28.8ms
Speed: 0.7ms preprocess, 28.8ms inference, 2.2ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 4 cars, 27.4ms
Speed: 1.1ms preprocess, 27.4ms inference, 3.1ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 4 cars, 22.7ms
Speed: 1.5ms preprocess, 22.7ms inference, 0.7ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 4 cars, 25.5ms
Speed: 0.7ms preprocess, 25.5ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 4 cars, 24.3ms
Speed: 0.5ms preprocess, 24.3ms inference, 1.6ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 4 cars, 24.4ms
Speed: 1.1ms preprocess, 24.4ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 4 cars, 25.3ms
Speed: 0.9ms preprocess, 25.3ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 224)

0: 160x

[81, 2]
[58, 10]
[51, 10]
[45, 10]
[44, 10]



0: 160x224 5 cars, 24.4ms
Speed: 2.8ms preprocess, 24.4ms inference, 1.3ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 23.4ms
Speed: 1.1ms preprocess, 23.4ms inference, 0.7ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 22.1ms
Speed: 0.8ms preprocess, 22.1ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 224)



...


0: 160x224 6 cars, 29.4ms
Speed: 1.3ms preprocess, 29.4ms inference, 1.3ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 6 cars, 25.0ms
Speed: 0.7ms preprocess, 25.0ms inference, 2.1ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 6 cars, 25.9ms
Speed: 0.8ms preprocess, 25.9ms inference, 1.9ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 6 cars, 22.4ms
Speed: 1.1ms preprocess, 22.4ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 6 cars, 26.3ms
Speed: 0.9ms preprocess, 26.3ms inference, 0.8ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 7 cars, 26.4ms
Speed: 0.7ms preprocess, 26.4ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 6 cars, 21.0ms
Speed: 0.6ms preprocess, 21.0ms inference, 0.7ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 8 cars, 25.4ms
Speed: 0.8ms preprocess, 25.4ms inference, 0.8ms postprocess per image at shape (1, 3, 160, 224)

0: 160x

[71, 17]
[72, 11]
[72, 11]


0: 160x224 7 cars, 23.5ms
Speed: 0.7ms preprocess, 23.5ms inference, 0.8ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 6 cars, 51.6ms
Speed: 14.5ms preprocess, 51.6ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 6 cars, 24.9ms
Speed: 1.9ms preprocess, 24.9ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 25.3ms
Speed: 1.1ms preprocess, 25.3ms inference, 1.2ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 29.4ms
Speed: 0.7ms preprocess, 29.4ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 28.1ms
Speed: 0.8ms preprocess, 28.1ms inference, 0.8ms postprocess per image at shape (1, 3, 160, 224)


[80, 6]



0: 160x224 5 cars, 28.4ms
Speed: 0.7ms preprocess, 28.4ms inference, 0.8ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 24.6ms
Speed: 0.7ms preprocess, 24.6ms inference, 1.8ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 26.7ms
Speed: 1.0ms preprocess, 26.7ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 26.2ms
Speed: 4.5ms preprocess, 26.2ms inference, 1.3ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 23.9ms
Speed: 0.9ms preprocess, 23.9ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 27.1ms
Speed: 1.5ms preprocess, 27.1ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 24.3ms
Speed: 0.6ms preprocess, 24.3ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 224)

0: 160x224 5 cars, 22.9ms
Speed: 1.2ms preprocess, 22.9ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 224)

0: 160x

### Display output video

In [13]:
tmp_output_path = "test_video.mp4"
output_path = "output_video.mp4"
subprocess.run(
    [
        "ffmpeg",
        "-i",
        tmp_output_path,
        "-crf",
        "18",
        "-preset",
        "veryfast",
        "-vcodec",
        "libx264",
        output_path,
        '-loglevel',
        'quiet'
    ]
)

CompletedProcess(args=['ffmpeg', '-i', 'test_video.mp4', '-crf', '18', '-preset', 'veryfast', '-vcodec', 'libx264', 'output_video.mp4', '-loglevel', 'quiet'], returncode=0)

In [14]:
!rm test_video.mp4

In [15]:
ipd.Video("output_video.mp4", width=600)